The code defines a simplified LangChain-like system using a custom Runnable abstract base class and concrete implementations for FakeLLM, FakePromptTemplate, FakeStrOutputParser, and a RunnableConnector. It demonstrates how these components can be chained together to create multi-step processing pipelines, mimicking LangChain's LCEL (LangChain Expression Language) for sequential operations.

In [1]:
from abc import ABC, abstractmethod # For defining Abstract Base Classes
import random # For generating random numbers

In [2]:
# Define an Abstract Base Class (ABC) named Runnable. 🛠️
# This sets the contract for any class that wants to be a "Runnable."
# Any class inheriting from Runnable MUST implement the 'invoke' method.
class Runnable(ABC):
    @abstractmethod # Decorator indicating that 'invoke' must be implemented by subclasses.
    def invoke(self, input_data): # 'self' is required for instance methods.
        pass

In [3]:
# Implement a FakeLLM class that conforms to the Runnable contract. 🤖
# This simulates a Language Model, returning random predefined responses.
class FakeLLM(Runnable):
    def __init__(self):
        print('LLM created') # Confirmation that the LLM instance is created.

    # The 'invoke' method is the core of a Runnable; it processes input_data.
    # Here, it takes a 'prompt' (which is expected to be a string from the previous step)
    # and returns a dictionary with a 'response' key, mimicking an LLM API's output.
    def invoke(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        # In a real LLM, this would send the prompt to an actual LLM service.
        return {'response': random.choice(response_list)}

    # The 'predict' method is redundant if 'invoke' is the primary interface.
    # It serves the same purpose as 'invoke' in this fake setup.
    def predict(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        return {'response': random.choice(response_list)}

In [4]:
# Implement a FakePromptTemplate class that conforms to the Runnable contract. 📝
# This simulates a prompt template that formats a string with given inputs.
class FakePromptTemplate(Runnable):
    def __init__(self, template, input_variables):
        self.template = template # The string template (e.g., 'Write a {length} poem about {topic}').
        self.input_variables = input_variables # Names of variables in the template.

    # The 'invoke' method processes the input dictionary to format the template.
    # It expects an 'input_dict' containing values for 'input_variables'.
    # It returns the formatted string, ready to be passed to an LLM.
    def invoke(self, input_dict):
        return self.template.format(**input_dict)

    # The 'format' method serves the same purpose as 'invoke' in this fake setup.
    def format(self, input_dict):
        return self.template.format(**input_dict)

In [5]:
# Implement a FakeStrOutputParser class that conforms to the Runnable contract. 📄
# This simulates a basic output parser that extracts a string from the LLM's raw output.
class FakeStrOutputParser(Runnable):
    def __init__(self):
        pass # No initialization needed for this simple parser.

    # The 'invoke' method takes the LLM's raw output (expected to be a dict with 'response' key)
    # and extracts just the string value associated with the 'response' key.
    def invoke(self, input_data):
        return input_data['response'] # Assumes input_data is a dictionary from FakeLLM.

In [6]:
# Implement a RunnableConnector class that acts as a chain. 🔗
# This class takes a list of Runnables and executes them sequentially.
# It mimics the sequential chaining functionality of LangChain Expression Language (LCEL).
class RunnableConnector(Runnable):
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list # A list of Runnable instances to execute in order.

    # The 'invoke' method processes input_data through each runnable in the list.
    # The output of one runnable becomes the input for the next.
    def invoke(self, input_data):
        current_data = input_data # Start with the initial input.
        for runnable in self.runnable_list:
            # Each runnable's 'invoke' method is called with the output of the previous one.
            current_data = runnable.invoke(current_data)
        return current_data # Return the final output after all runnables have executed.

In [7]:
# --- First Chain Example ---

# Define a prompt template for generating a poem.
template = FakePromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [8]:
# Initialize the fake LLM and parser.
llm = FakeLLM()
parser = FakeStrOutputParser()

LLM created


In [9]:
# Create a chain using RunnableConnector. ⛓️
# This chain will:
# 1. Take initial input (e.g., {'length':'long', 'topic':'paris'}).
# 2. Pass it to 'template' (FakePromptTemplate) to format the prompt.
# 3. Pass the formatted prompt to 'llm' (FakeLLM) to get a fake LLM response (dict).
# 4. Pass the LLM response to 'parser' (FakeStrOutputParser) to extract the string content.
chain = RunnableConnector([template, llm, parser])

In [10]:
# Invoke the first chain. 🚀
# This executes the entire pipeline defined by 'chain'.
result1 = chain.invoke({'length':'long', 'topic':'paris'})
print(f"First Chain Result: {result1}") # Will print one of the random LLM responses.

First Chain Result: Delhi is the capital of India


In [11]:
# --- Second (Nested) Chain Example ---

# Define templates for a joke and its explanation.
template1 = FakePromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

template2 = FakePromptTemplate(
    template='Explain the following joke {response}', # Note: 'response' matches LLM output key.
    input_variables=['response']
)

In [12]:
# Initialize LLM and parser again (can reuse instances).
llm = FakeLLM()
parser = FakeStrOutputParser()

LLM created


In [13]:
# Create 'chain1': Generates a joke. ⛓️
# It takes a 'topic', formats it into a joke prompt, and gets a joke from the LLM.
# The output of chain1 will be a dictionary like {'response': 'IPL is a cricket league'} (from FakeLLM)
# because the parser isn't at the end of chain1. This output structure is crucial for template2.
chain1 = RunnableConnector([template1, llm])

# Create 'chain2': Explains a joke. ⛓️
# It takes the LLM's raw response (e.g., {'response': 'IPL is a cricket league'})
# formats it into an explanation prompt using `template2` (which expects a 'response' key),
# gets an explanation from the LLM, and then parses it into a string.
chain2 = RunnableConnector([template2, llm, parser])

In [14]:
# Create 'final_chain': A nested chain combining chain1 and chain2. ⛓️🔗
# 1. Takes initial input (e.g., {'topic':'cricket'}).
# 2. Passes it to 'chain1', which generates a joke (as a dictionary like {'response': '...'}).
# 3. The output of 'chain1' (the dictionary) is then passed as input to 'chain2'.
#    Crucially, 'chain2' expects a dictionary input that contains a 'response' key,
#    which aligns perfectly with 'chain1's output.
# 4. 'chain2' then explains the joke and extracts the final string.
final_chain = RunnableConnector([chain1, chain2])

In [15]:
# Invoke the final nested chain. 🚀
result2 = final_chain.invoke({'topic':'cricket'})
print(f"Second Chain Result: {result2}") # Will print the 'explanation' (another random LLM response).

Second Chain Result: AI stands for Artificial Intelligence
